In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import geoplot as gplt

ModuleNotFoundError: No module named 'geoplot'

In [7]:
def make_polygon_coords (xcenter, ycenter, xdelta, ydelta):
    return [
        [xcenter - xdelta, ycenter - ydelta],
        [xcenter + xdelta, ycenter - ydelta],
        [xcenter + xdelta, ycenter + ydelta],
        [xcenter - xdelta, ycenter + ydelta],
        [xcenter - xdelta, ycenter - ydelta]
    ]


In [8]:
make_polygon_coords(.5, .5, .5, .5)

[[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0], [0.0, 0.0]]

In [44]:
def make_geoJson_from_bins (bin_counts, xedges, yedges):
    
    def make_polygon_coords (xcenter, ycenter, xdelta, ydelta):
        return [[
            [xcenter - xdelta, ycenter - ydelta],
            [xcenter + xdelta, ycenter - ydelta],
            [xcenter + xdelta, ycenter + ydelta],
            [xcenter - xdelta, ycenter + ydelta],
            [xcenter - xdelta, ycenter - ydelta]
        ]]
    """
    Using the output from numpy.histogram2d, create a geoJSON with geometry features of Polygons
    and properties features of the bin_counts.  This is used to create a gridded choropleth with 
    finer detail.
    """
    m = len(bin_counts)
    n = len(bin_counts[0])
    xcenter = (xedges[:-1] + xedges[1:]) / 2
    ycenter = (yedges[:-1] + yedges[1:]) / 2
    x_delta = xcenter[0] - xedges[0]
    y_delta = ycenter[0] - yedges[0]
    
    features = []
    for j in range(n):
        for i in range(m):
            poly = make_polygon_coords(xcenter[i], ycenter[j], x_delta, y_delta)
            bincount = bin_counts[j, i]
            
            feature = {"type": "Feature",
                      "properties": {
                          "count": bincount
                      },
                      "geometry": {
                          "type": "Polygon",
                          "coordinates": poly
                      }}
            features.append(feature)
            
            print(feature)
    feature_collection = {
        "type": "FeatureCollection",
        "features": features
    }
    return feature_collection


In [50]:
def make_geoJson_bincounts_from_coords(lon, lat, bins):
    """
    bins can be any form that np.histogram2d accepts as a bins argument.
    """
    H, xedges, yedges = np.histogram2d(lon, lat, bins=bins)
    H = H.T
    return make_geoJson_from_bins(H, xedges, yedges)


In [13]:
x = np.random.randint(low = 0, high = 5, size=10)
y = np.random.randint(low = 0, high = 5, size=10)

for pair in zip(x,y):
    print(pair)

(0, 0)
(1, 4)
(4, 2)
(4, 3)
(4, 3)
(1, 0)
(1, 1)
(2, 2)
(0, 3)
(0, 3)


In [38]:
H, xedges, yedges = np.histogram2d(x, y, [4, 4])

In [39]:
H = H.T

In [40]:
print(H, xedges, yedges)

[[1. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 1.]
 [2. 1. 0. 2.]] [0. 1. 2. 3. 4.] [0. 1. 2. 3. 4.]


In [45]:
make_geoJson_from_bins(H, xedges, yedges)

{'type': 'Feature', 'properties': {'count': 1.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0], [0.0, 0.0]]]}}
{'type': 'Feature', 'properties': {'count': 1.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[[1.0, 0.0], [2.0, 0.0], [2.0, 1.0], [1.0, 1.0], [1.0, 0.0]]]}}
{'type': 'Feature', 'properties': {'count': 0.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[[2.0, 0.0], [3.0, 0.0], [3.0, 1.0], [2.0, 1.0], [2.0, 0.0]]]}}
{'type': 'Feature', 'properties': {'count': 0.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[[3.0, 0.0], [4.0, 0.0], [4.0, 1.0], [3.0, 1.0], [3.0, 0.0]]]}}
{'type': 'Feature', 'properties': {'count': 0.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[[0.0, 1.0], [1.0, 1.0], [1.0, 2.0], [0.0, 2.0], [0.0, 1.0]]]}}
{'type': 'Feature', 'properties': {'count': 1.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[[1.0, 1.0], [2.0, 1.0], [2.0, 2.0], [1.0, 2.0], [1.0, 1.0]]]}}
{'type': 'Feature', 'propert

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'count': 1.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[0.0, 0.0],
      [1.0, 0.0],
      [1.0, 1.0],
      [0.0, 1.0],
      [0.0, 0.0]]]}},
  {'type': 'Feature',
   'properties': {'count': 1.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[1.0, 0.0],
      [2.0, 0.0],
      [2.0, 1.0],
      [1.0, 1.0],
      [1.0, 0.0]]]}},
  {'type': 'Feature',
   'properties': {'count': 0.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[2.0, 0.0],
      [3.0, 0.0],
      [3.0, 1.0],
      [2.0, 1.0],
      [2.0, 0.0]]]}},
  {'type': 'Feature',
   'properties': {'count': 0.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[3.0, 0.0],
      [4.0, 0.0],
      [4.0, 1.0],
      [3.0, 1.0],
      [3.0, 0.0]]]}},
  {'type': 'Feature',
   'properties': {'count': 0.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[0.0, 1.0],
      [1.0, 1.0],
      [1.0, 2.0],
   

In [51]:
make_geoJson_bincounts_from_coords(x, y, [4,4])

{'type': 'Feature', 'properties': {'count': 1.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0], [0.0, 0.0]]]}}
{'type': 'Feature', 'properties': {'count': 1.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[[1.0, 0.0], [2.0, 0.0], [2.0, 1.0], [1.0, 1.0], [1.0, 0.0]]]}}
{'type': 'Feature', 'properties': {'count': 0.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[[2.0, 0.0], [3.0, 0.0], [3.0, 1.0], [2.0, 1.0], [2.0, 0.0]]]}}
{'type': 'Feature', 'properties': {'count': 0.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[[3.0, 0.0], [4.0, 0.0], [4.0, 1.0], [3.0, 1.0], [3.0, 0.0]]]}}
{'type': 'Feature', 'properties': {'count': 0.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[[0.0, 1.0], [1.0, 1.0], [1.0, 2.0], [0.0, 2.0], [0.0, 1.0]]]}}
{'type': 'Feature', 'properties': {'count': 1.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[[1.0, 1.0], [2.0, 1.0], [2.0, 2.0], [1.0, 2.0], [1.0, 1.0]]]}}
{'type': 'Feature', 'propert

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'count': 1.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[0.0, 0.0],
      [1.0, 0.0],
      [1.0, 1.0],
      [0.0, 1.0],
      [0.0, 0.0]]]}},
  {'type': 'Feature',
   'properties': {'count': 1.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[1.0, 0.0],
      [2.0, 0.0],
      [2.0, 1.0],
      [1.0, 1.0],
      [1.0, 0.0]]]}},
  {'type': 'Feature',
   'properties': {'count': 0.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[2.0, 0.0],
      [3.0, 0.0],
      [3.0, 1.0],
      [2.0, 1.0],
      [2.0, 0.0]]]}},
  {'type': 'Feature',
   'properties': {'count': 0.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[3.0, 0.0],
      [4.0, 0.0],
      [4.0, 1.0],
      [3.0, 1.0],
      [3.0, 0.0]]]}},
  {'type': 'Feature',
   'properties': {'count': 0.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[0.0, 1.0],
      [1.0, 1.0],
      [1.0, 2.0],
   

In [19]:
len(H)

10

In [20]:
len(H[0])

10

In [27]:
for yrow in H:
    print(yrow)

[1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[2. 0. 0. 0. 0. 0. 0. 0. 0. 2.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [28]:
H[0]

array([1., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [29]:
H[0,2]

1.0

In [31]:
H[0,1]

0.0

In [47]:
np.histogram2d?